In [ ]:
import numpy as np
import tensorflow as tf
import random
from numpy.core.fromnumeric import argmax

In [ ]:
x = []
y = []

test_case = 700

for i in range(test_case) :
  a = [random.randint(0,9),random.randint(0,9),random.randint(0,9)]
  b = [a[2], a[1], a[0]]
  x.append(a)
  y.append(b)

In [ ]:
encoder_input = np.zeros((test_case, 3, 10))
decoder_input = np.zeros((test_case, 3, 10))
decoder_output = np.zeros((test_case, 3, 10))

In [ ]:
for i in range(test_case) :
  for j in range(3) :
    encoder_input[i][j][x[i][j]] = 1
    decoder_output[i][j][x[i][j]] = 1
    if j :
      decoder_input[i][j][x[i][j-1]] = 1

In [ ]:
encoder_layer_input = tf.keras.layers.Input(shape=(None,10))
encoder_layer_lstm = tf.keras.layers.LSTM(20, return_state=True)
encoder_lstm_output, encoder_lstm_h, encoder_lstm_c = encoder_layer_lstm(encoder_layer_input)
encoder_lstm_states = [encoder_lstm_h, encoder_lstm_c]

decoder_layer_input = tf.keras.layers.Input(shape=(None,10))
decoder_layer_lstm = tf.keras.layers.LSTM(20, return_sequences=True, return_state=True)
decoder_lstm_output, decoder_lstm_h, decoder_lstm_c = decoder_layer_lstm(decoder_layer_input, initial_state=encoder_lstm_states)

decoder_dense = tf.keras.layers.Dense(10, activation='softmax')
decoder_dense_output = decoder_dense(decoder_lstm_output)

In [ ]:
model = tf.keras.Model(inputs=[encoder_layer_input, decoder_layer_input], outputs=decoder_dense_output)
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input, decoder_input], decoder_output, epochs=700)

In [ ]:
encoder = tf.keras.Model(encoder_layer_input, encoder_lstm_states)

In [ ]:
decoder_hidden = tf.keras.layers.Input(shape=(20))
decoder_cell = tf.keras.layers.Input(shape=(20))
decoder_input_states = [decoder_hidden, decoder_cell]

decoder_lstm_output, decoder_lstm_h, decoder_lstm_c = decoder_layer_lstm(decoder_layer_input, initial_state=decoder_input_states)
decoder_dense_output = decoder_dense(decoder_lstm_output)
decoder_states = [decoder_lstm_h, decoder_lstm_c]

In [ ]:
decoder = tf.keras.Model(inputs= [decoder_layer_input]+decoder_input_states, outputs= [decoder_dense_output]+decoder_states)

In [ ]:
def get_prediction(i) :
  test_encoder_input = np.array([encoder_input[i]])
  test_encoder_output = encoder.predict(test_encoder_input)
  test_decoder_input = np.zeros((1,1,10))

  ans = []

  for i in range(3) :

    test_decoder_output, test_decoder_h, test_decoder_c = decoder.predict([test_decoder_input] + test_encoder_output)

    val = argmax(test_decoder_output)
    test_encoder_output = [test_decoder_h, test_decoder_c]
    test_decoder_input = np.zeros((1,1,10))
    test_decoder_input[0][0][val] = 1
    ans.append(val)

  ans.reverse()

  return ans

In [ ]:
i = 20
print(y[i])
print(get_prediction(i))

[2, 3, 6]
[2, 3, 6]


In [ ]:
for i in range(200) :
  result = get_prediction(i)
  print(x[i] , '\t', result, '\t', y[i]==result)